# Criar Banco de Dados

Execute tudo de uma vez

In [ ]:
import pandas as pd
import pyodbc
from flask_bcrypt import generate_password_hash

Edite a ConnectionString se necessário

In [ ]:
SERVER = r'.\SQLEXPRESS'
DRIVER = 'ODBC Driver 17 for SQL Server'
CONNECTIONSTRING = f'DRIVER={DRIVER};SERVER={SERVER};Trusted_Connection=yes;'
CONN = pyodbc.connect(CONNECTIONSTRING, autocommit = True)
CURSOR = CONN.cursor()

In [ ]:
CURSOR.execute(''' DROP DATABASE IF EXISTS dbOneplay ''')
CURSOR.execute(''' CREATE DATABASE dbOneplay ''')

In [ ]:
CURSOR.execute(''' USE dbOneplay ''')

In [ ]:
TABELAS = {}
TABELAS['tblUsuarios'] = ('''
    create table tblUsuarios(
	UsuarioID int not null identity(1,1) primary key,
	Nome varchar(max) not null,
	Senha varchar(max) not null,
	OP bit default 0,
	VersaoDados datetime null,)                   
''')

TABELAS['tblJogos'] = ('''
    create table tblJogos(
	JogoID int not null identity(1,1) primary key,
	CategoriaID int,
	ExibirHome bit default 0,
	Ordem int,
	Nome varchar(max),
	VersaoDados datetime null,)
''')

TABELAS['tblCategorias'] = ('''
    create table tblCategorias(
	CategoriaID int not null identity(1,1) primary key,
	Categoria varchar(max) not null,
	VersaoDados datetime null,)
''')

TABELAS['tblPaginaJogos'] = ('''
    create table tblPaginaJogos(
	PaginaJogoID int not null identity(1,1) primary key,
	JogoID int not null,
	ResumoJogo varchar(max),
	LinkDownload varchar(max),
	DescricaoJogo varchar(max),
	TituloTutorial varchar(max),
	DescricaoTutorial varchar(max),
	TituloNoticia1 varchar(max),
	DescricaoNoticia1 varchar(max),
	TituloNoticia2 varchar(max),
	DescricaoNoticia2 varchar(max),
	TituloVideo varchar(max),
	DescricaoVideo varchar(max),
	LinkVideo varchar(max),
	VersaoDados datetime null,)
''')

TABELAS['tblImagensJogos'] = ('''
    create table tblImagensJogos(
	ImagemJogoID int not null identity(1,1) primary key,
	JogoID int ,
	ImgIndex varbinary(max),
	ImgPaginaJogo varbinary(max),
	ImgNoticia1 varbinary(max),
	ImgNoticia2 varbinary(max),
	VersaoDados datetime null,)               
''')

TABELAS['tblTutorialJogos'] = ('''
    create table tblTutorialJogos(
	TutorialJogoid int not null identity(1,1) primary key,
	JogoID int ,
	Ordem int,
	Passo varchar(max),
	DescricaoPasso varchar(max),
	VersaoDados datetime null,)                 
''')

TABELAS['TRIGGER_tblUsuarios'] = ('''
	CREATE TRIGGER Versao_Dados_tblUsuarios
	ON tblUsuarios
	AFTER UPDATE,INSERT
	AS
	BEGIN
		UPDATE tblUsuarios SET VersaoDados = GETDATE() WHERE UsuarioID IN (SELECT DISTINCT UsuarioID FROM inserted)
	END;
''')

TABELAS['TRIGGER_tblJogos'] = ('''
	CREATE TRIGGER Versao_Dados_tblJogos
	ON tblJogos
	AFTER UPDATE,INSERT
	AS
	BEGIN
		UPDATE tblJogos SET VersaoDados = GETDATE() WHERE JogoID IN (SELECT DISTINCT JogoID FROM inserted)
	END;
''')

TABELAS['TRIGGER_tblCategorias'] = ('''
	CREATE TRIGGER Versao_Dados_tblCategorias
	ON tblCategorias
	AFTER UPDATE,INSERT
	AS
	BEGIN
		UPDATE tblCategorias SET VersaoDados = GETDATE() WHERE CategoriaID IN (SELECT DISTINCT CategoriaID FROM inserted)
	END;
''')

TABELAS['TRIGGER_tblPaginaJogos'] = ('''
	CREATE TRIGGER Versao_Dados_tblPaginaJogos
	ON tblPaginaJogos
	AFTER UPDATE,INSERT
	AS
	BEGIN
		UPDATE tblPaginaJogos SET VersaoDados = GETDATE() WHERE PaginaJogoID IN (SELECT DISTINCT PaginaJogoID FROM inserted)
	END;
''')

TABELAS['TRIGGER_tblImagensJogos'] = ('''
	CREATE TRIGGER Versao_Dados_tblImagensJogos
	ON tblImagensJogos
	AFTER UPDATE,INSERT
	AS
	BEGIN
		UPDATE tblImagensJogos SET VersaoDados = GETDATE() WHERE ImagemJogoID IN (SELECT DISTINCT ImagemJogoID FROM inserted)
	END;
''')

TABELAS['TRIGGER_tblTutorialJogos'] = ('''
	CREATE TRIGGER Versao_Dados_tblTutorialJogos
	ON tblTutorialJogos
	AFTER UPDATE,INSERT
	AS
	BEGIN
		UPDATE tblTutorialJogos SET VersaoDados = GETDATE() WHERE JogoID IN (SELECT DISTINCT JogoID FROM inserted)
	END;
''')

TABELAS['ForeignKeys'] = ('''
    ALTER TABLE tblPaginaJogos ADD CONSTRAINT FK_tblBanners_tblJogos_JogoID FOREIGN KEY(JogoID) REFERENCES tblJogos(JogoID)
	ALTER TABLE tblImagensJogos ADD CONSTRAINT FK_tblImagens_tblJogos_JogoID FOREIGN KEY(JogoID) REFERENCES tblJogos(JogoID)
	ALTER TABLE tblTutorialJogos ADD CONSTRAINT FK_tblTutoriais_tblJogos_JogoID FOREIGN KEY(JogoID) REFERENCES tblJogos(JogoID)
	ALTER TABLE tblJogos ADD CONSTRAINT FK_tblJogos_tblCategorias_CategoriaID FOREIGN KEY(CategoriaID) REFERENCES tblCategorias(CategoriaID)
''')

for tabela_nome in TABELAS:
    tabela_sql = TABELAS[tabela_nome]
    try:
        print('\nCriando {}:'.format(tabela_nome))
        CURSOR.execute(tabela_sql)
    except pyodbc.Error as ex:
        sqlstate = ex.args[0] 
        if sqlstate == '42000':
            print("Erro de sintaxe SQL.")
        else:
            print(f"Ocorreu um erro: {ex}")
        exit()

In [ ]:
planilhas = pd.ExcelFile('PreCadastro.xlsx')
nomes_planilhas = planilhas.sheet_names
dados = {sheet: planilhas.parse(sheet) for sheet in nomes_planilhas}
dados['tblJogos']

In [ ]:
dados['tblJogos'].iloc[0].iloc[3]

In [ ]:
print(len(dados['tblJogos']))
print(len(dados['tblJogos'].iloc[0]))
print(len(dados['tblJogos'].iloc[0].iloc[3]))

In [ ]:
CURSOR.execute(f'''insert into tblUsuarios(Nome,Senha,OP) values('admin','{generate_password_hash('%1234').decode('utf-8')}',1)''')

In [ ]:
CURSOR.executemany('insert into tblCategorias(Categoria) values(?)',dados['tblCategorias'].values.tolist())

In [ ]:
CURSOR.executemany('insert into tblJogos(CategoriaID,ExibirHome,Ordem,Nome) values(?,?,?,?)',dados['tblJogos'].values.tolist())

In [ ]:
CURSOR.executemany('insert into tblTutorialJogos(JogoID,Ordem,Passo,DescricaoPasso) values(?,?,?,?)',dados['tblTutorialJogos'].values.tolist())

In [ ]:
CURSOR.executemany('insert into tblPaginaJogos(JogoID,ResumoJogo,LinkDownload,DescricaoJogo,TituloTutorial,DescricaoTutorial,TituloNoticia1,DescricaoNoticia1,TituloNoticia2,DescricaoNoticia2,TituloVideo,DescricaoVideo,LinkVideo) values(?,?,?,?,?,?,?,?,?,?,?,?,?)',dados['tblPaginaJogos'].values.tolist())

In [ ]:
CURSOR.execute('''select * from tblJogos''').fetchall()

In [ ]:
CURSOR.close()
CONN.close()